In [ ]:
from huggan.pytorch.metrics.fid_score import calculate_fretchet
from huggan.pytorch.metrics.inception import InceptionV3

from PIL import Image
from torchvision.transforms.functional import to_tensor
import torch
from torchvision.utils import make_grid

In [ ]:
from demo import load_model,generate,get_train_data

In [ ]:
calc_size=100
model= InceptionV3()

In [ ]:
dataset=get_train_data()
images_real = [to_tensor(ex.resize((512,512))) for ex in dataset["image"][:calc_size]]
images_real = torch.stack(images_real)

In [ ]:
import wandb

In [ ]:
model_name='ceyda/butterfly_cropped_uniq1K_512'
# git rev-list main --first-parent
versions=[
    "95a9596a1e47e2419c9bd5252d809eecb14fdcf4"
"0738c09d1c4bde9bad9ff1800c639d9d50abb2d8",
"e30e46e5ecbdc08aa7817aad35f468b34fc6aa43",
"a61360529d4ce4b848b9dcf71f3100f8f0746630",
"9af3333573664cc8654f6643f373d1ac0d769173",
"406edbcc87643ae935238f430a21309e646b8c31",
"54a7a0103796a4d6a242d2fdddad40814f9f48ba",
"0edac54b81958b82ce9fd5c1f688c33ac8e4f223",
"4aa71c62b3bcd4fe0d79190a069e0a16fe84838d",
"3f3fe29048a5ce3de7b0cb2c1a03194a123ef276",
"57d36a15546909557d9f967f47713236c8288838",
"ed35ab2acdf9281cc2d38a136ce76cc714e474d8",
"94a20acf6ea4fbadca04baaa62cdbe3d77823040",
"afeb8691aa04c4114a7c4a323a9b2d3938f13995",
"4ed1c58c2db77e7b68a9737261046959b4e2a5ca",
"5498b1de2c523a70f89a8a7f36f344a7938ec53c",
"ea571e51e052025f5a3bc780c86589530eaf0e7c",
"77ca720fab77439ecea6b8c98c284c4569482741",
"7c0344c29455eedfd50c9f44b04e2a7d28bb7cf8",
"711389bf73d0be68065032a01efb82c937e44ea1",
"0d0e100d32f505a74edc788d6e21a43d17baf2e4",
"f8f1f001f031c0f72519fdfb065eccd8b730aaf0",
"fd170b285c658f271ae73632a55c12d1ff6590da",
"e41125be4b1994516cf56eee966af252a17b3e2e",
"fc57f489f38b60a52b9d197439466e549ebed40a",
"9265be8a24a3a1fa95fc9533673cf0fbb1abc3f5"
]

In [ ]:
for v,version in enumerate(versions):
    run=wandb.init(project='butterfly-gan-fid',reinit=True,name=f"{v}-{version}",resume=True)
    # model_name='ceyda/butterfly_512_base'
    gan = load_model(model_name,version)
    batch_size=4
    images_fake = []
    for e in range(int(calc_size/batch_size)):
        gs = generate(gan,batch_size)
        for i,im in enumerate(gs):
            img=Image.fromarray(im)
            # img.save(f"./assets/outputs/{e*batch_size+i}_fake.jpg")
            images_fake.append(to_tensor(img))
    # images_fake = [to_tensor(Image.open(f"./assets/outputs/{i}_fake.jpg")) for i in range(calc_size)]
    images_fake = torch.stack(images_fake)
    score=calculate_fretchet(images_real, images_fake, model)
    print(f"{v}-{version}: {score}")
    wandb.log({"examples": wandb.Image(make_grid(images_fake,nrow=4))})
    wandb.log({"fid_score":score})
    run.finish()


In [ ]:
calculate_fretchet(images_real, images_fake, model)